# Problemas de alocação

Os problemas de alocação modelam uma relação entre dois tipos de entidades: *compromissos* e *recursos*. Adicionalmente, as restrições sobre a alocação de recursos a compromissos classificam-se em dois tipos: *obrigações* e *limitações*.

Nesta aula vamos considerar um problema de alocação. Pretende-se que faça a modelação do problema em Programação Inteira e que use o SCIP para os resolver.
Usaremos a biblioteca [OR-Tools](https://developers.google.com/optimization) para fazer a interface para o SCIP. 

## Horário de um centro de estudos

Um centro de estudos possui $S$ salas de aula, $P$ professores, e está aberto $H$ horas por dia, durante $D$ dias na semana.

O centro de estudos funciona com as seguintes regras:
- Cada professor não pode dar mais do que $C$ horas por dia.
- Todos os professores do centro devem ter uma carga horária semanal parecida (diferença máxima de 2 horas).
- Não é permitido mais do que um professor por sala.
- Alocações contíguas do mesmo professor têm de ser na mesma sala.

Pretende-se estabelecer um horário para o centro de estudos que permita ter um número mínimo de aulas $N$ por semana.

### Análise do problema

Este é um problema de alocação. Pretende-se alocar professores a salas de aula, ao longo da semana, sendo o tempo de ocupação das salas de uma hora. 

Existem $P$ professores, que podemos identificar por um índice $p \in [0..P\!-\!1]$, e podemos identificar cada sala disponível num dado dia, a uma dada hora, por um triplo $(s,d,h) \in [0..S\!-\!1]\times[0..D\!-\!1]\times[0..H\!-\!1]$.

Vamos usar uma família $x_{p,s,d,h}$ de variáveis binárias (i.e., que assumem valores inteiros $\{0,1\}$), com a seguinte semântica

$$x_{p,s,d,h} == 1  {se e só se}$$ o professor $p$ for alocado à sala $s$, no dia $d$, à hora 

Estas $P\times S\times D\times H$ variáveis são convenientemente representadas numa matriz $X$ instanciável com valores $\{0,1\}^{P\times S\times D\times H}$, a que se costuma chamar *matriz de alocação*.

Destaca-se ainda o seguinte:

**Limitações** (que impõem limites máximos à alocação)

1. A carga horária diária máxima de cada professor é $C$.
2. Cada sala tem alocado, no máximo, um professor.
3. Em cada dia e hora, cada professor é alocado 0 ou 1 vezes.
4. No mesmo dia duas alocações contíguas do mesmo professor têm de ser na mesma sala.


**Obrigações** (que impõem limites mínimos à alocação)

5. Todos os professores têm uma carga horaria semanal parecida (diferença máxima de 2 horas).
6. O número mínimo de aulas por semana é $N$.

### Implementação

Começamos por importar a biblioteca de programação linear do OR-Tools e criar uma instância do *solver*.


Depois inicializamos o *solver* `horario` e definir os valores para as constantes $S$, $P$, $H$, $D$, $C$ e $N$.

In [112]:
from ortools.linear_solver import pywraplp

horario = pywraplp.Solver.CreateSolver('SCIP')



S, P, H, D, C, N = 2, 2, 6, 5, 4, 21

Em seguida, declaramos a matriz de alocação, $X$, e uma variável $T$ que irá representar a carga horária semanal mínima de cada professor (com as restrições adequadas).

### Exercício 1

Complete a declaração da matriz de alocação $X$ como um dicionário.

In [113]:
X = {}

for p in range(P):
    for s in range(S):
        for d in range(D):
            for h in range(H):
                X[p,s,d,h] = horario.IntVar(0,1,f"x[{p},{s},{d},{h}]") 



    

T = horario.IntVar(0.0,H*D,"T")

Passamos agora à modelação das restrições e à sua introdução no *solver*.

A restrição

1. A carga horária diaria máxima de cada professor é $C$

pode expressar-se da seguinte forma:

$$\forall_{d< D} \cdot \forall_{p< P} \cdot \quad \sum_{h< H,\,s< S} x_{p,s,d,h} \leq C$$

In [114]:
for d in range(D):
    for p in range(P):
        horario.Add(sum([X[p,s,d,h] for h in range(H) for s in range(S)]) <= C)

### Exercício 2

Apresente as fórmulas que modelam as restantes restrições e acrescente-as ao problema `horario`.

2. Cada sala tem alocada, no máximo, um professor.

$$\forall_{d< D} \cdot \forall_{h< H} \cdot \forall_{s< S} \cdot \quad \sum_{p< P} x_{p,s,d,h} \leq 1$$

In [115]:
for d in range(D):
    for h in range(H):
        for s in range(S):
            horario.Add(sum([X[p,s,d,h] for p in range(P)]) <= 1)

3. Em cada dia e hora, cada professor é alocado 0 ou 1 vezes.

$$ (completar) $$

In [116]:
for p in range(P):
    for h in range(H):
        for d in range(D):
            horario.Add(sum([X[p,s,d,h] for s in range(S)]) <= 1)

4. Todos os professores têm uma carga horaria semanal parecida (diferença máxima de 2 horas).

$$ (completar) $$

In [117]:
for p1 in range(P):
    for p2 in range(p1+1,P):
        horario.Add(sum([X[p1,s,d,h] for s in range(S) for d in range(D) for h in range(H)]) - sum([X[p2,s,d,h] for s in range(S) for d in range(D) for h in range(H)]) <= 2)
        horario.Add(sum([X[p2,s,d,h] for s in range(S) for d in range(D) for h in range(H)]) - sum([X[p1,s,d,h] for s in range(S) for d in range(D) for h in range(H)]) <= 2)

5. No mesmo dia duas alocações contíguas do mesmo professor têm de ser na mesma sala.

$$ (completar) $$

6. O número mínimo de aulas por semana é $N$.

$$
(completar)
$$

horario.Add(sum([X[p,s,d,h] for d in range(D) for h in range(H) for s in range(S) for p in range(P)]) >= N)

### Exercício 3

Finalize a resolução do problema procurando uma solução, e imprimindo o número de aulas por semana, caso exista.

In [120]:
stat = horario.Solve()

if stat == pywraplp.Solver.OPTIMAL:
    total_alocacoes = 0
    print("\n========== SOLUÇÃO ÓTIMA ENCONTRADA ==========\n")

    for s in range(S):  # sala
        print(f"SALA {s}")
        print("-" * 60)
        tem_aula = False

        # Cabeçalho de colunas (horários)
        print(f"{'Dia':<10} {'Hora':<10} {'Professor':<10}")
        print("-" * 60)

        for d in range(D):  # dia
            for h in range(H):  # horário
                linha_vazia = True
                for p in range(P):  # professor
                    if X[p, s, d, h].solution_value() == 1:
                        total_alocacoes += 1
                        tem_aula = True
                        linha_vazia = False
                        print(f"{d:<10} {h:<10} {p:<10}")
                if linha_vazia:
                    # Marca horário livre
                    print(f"{d:<10} {h:<10} {'(vazio)':<10}")

        if not tem_aula:
            print("(Nenhuma aula atribuída nesta sala)")

        print("=" * 60)
        print()

    print(f"TOTAL DE ALOCAÇÕES: {total_alocacoes}")
    print("==============================================\n")

else:
    print("Não foi possível encontrar uma solução ótima.")



========== SOLUÇÃO ÓTIMA ENCONTRADA ==========

SALA 0
------------------------------------------------------------
Dia        Hora       Professor 
------------------------------------------------------------
0          0          0         
0          1          0         
0          2          0         
0          3          0         
0          4          1         
0          5          1         
1          0          0         
1          1          0         
1          2          0         
1          3          0         
1          4          1         
1          5          1         
2          0          0         
2          1          0         
2          2          0         
2          3          0         
2          4          1         
2          5          1         
3          0          0         
3          1          0         
3          2          0         
3          3          0         
3          4          1         
3          5          1       

### Exercício 4

Defina funções para construir os horarios de professores e salas individuais, e para apresentar de forma legível esses horários.

In [3]:
# completar

### Exercício 5

Explore o comportamento do modelo pela variação dos parâmetros $S$, $P$, $H$, $D$, $C$ e $N$.

### Exercício 6

Queremos agora acrescentar a seguinte regra no funcionamento do centro de estudos:

> Cada professor tem de ter um dia da semana em que não dá aulas.

Esta *obrigação* poderia ser expressa matematicamente, de forma direta, por
$$
\forall_{p<P}.\exists_{d<D}. \quad \sum_{s<S,h<H} x_{p,s,d,h} = 0
$$
ou, em alternativa, pela seguinte expressão
$$
\forall_{p<P}. \quad \bigvee_{d<D} \big(\sum_{s<S,h<H} x_{p,s,d,h} = 0\big) 
$$

Contudo a disjunção não tem uma representação direta nos solvers para programação inteira. Para a implementar podemos acrescentar uma família de variáveis binárias $y_{p,d}$ que indicam se o professor $p$ dá aulas no dia $d$, com a seguinte restrição que limita o número máximo de dias em que o professor dá aulas.

$$\forall_{p<P} \cdot \quad \sum_{d<D} y_{p,d} \leq D-1 $$

O valor de  $y_{p,d}$ deve também de alguma forma limitar superiormente as aulas que o professor $p$ dá no dia $d$.
Apresente uma fórmula que modele esta nova limitação.

$$\forall_{p<P} \cdot \forall_{d<D} \cdot \quad \sum_{s<S,h<H} x_{p,s,d,h} \leq y_{p,d} \times H$$



Acrescente agora estas fórmulas ao problema `horario`, e encontre nova solução.

In [4]:
# completar